In [ ]:
%load_ext autoreload
%autoreload 2


from pathlib import Path
from cellpose import plot, models, core, utils, io, dynamics
import skimage.io
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
import time, os, sys
from tifffile import imread
# use_GPU = core.use_gpu()
# print('>>> GPU activated? %d'%use_GPU)

import omnipose

In [ ]:
base = '/home/kcutler/DataDrive/omnipose_all'
# base = '/home/kcutler/DataDrive/omnipose_all/fluor'

# train_dir = os.path.join(base,'train_sorted')
# test_dir = '/home/kcutler/DataDrive/omnipose_test'
# test_dir = os.path.join(base,'test_sorted')
# basedir = [train_dir,test_dir]
subset = ['train','test']
basedir = [os.path.join(base,s + '_sorted') for s in subset]

def getname(path,suffix='_masks'):
    return os.path.splitext(Path(path).name)[0].replace(suffix,'')

def get_GT(basedir):
    mask_filter = '_masks'
    img_names = io.get_image_files(basedir,mask_filter,look_one_level_down=True)
    mask_names,flow_names = io.get_label_files(img_names, mask_filter)
    #annoyingly, the names are not sorted properly because the paths are all different for the test folder...
    mask_names = sorted(mask_names,key=getname)
    img_names = sorted(img_names,key=getname)
    imgs = [skimage.io.imread(f) for f in img_names]
    # masks_gt = [utils.format_labels(utils.clean_boundary(skimage.io.imread(f))) for f in mask_names]
    masks_gt = [omnipose.utils.format_labels(omnipose.utils.clean_boundary(skimage.io.imread(f))) for f in mask_names]

    return imgs, masks_gt, img_names, mask_names


imgs =  [[]]*2
masks_gt =  [[]]*2
img_names =  [[]]*2
mask_names =  [[]]*2
for k in range(2):
    imgs[k], masks_gt[k], img_names[k], mask_names[k] = get_GT(basedir[k])
# train_imgs, train_masks, train_img_names, train_mask_names = get_GT(train_dir)
# test_imgs, test_masks, test_img_names, train_mask_names = get_GT(train_dir)

### Count images and cells in dataset


In [ ]:
# Count images
import fastremap, ncolor
from matplotlib.colors import ListedColormap
from omnipose import utils
from cellpose import plot
cmap = ListedColormap([color for color in utils.sinebow(5).values()])

imagecount = 0
cellcount = 0
show = 0
for k in range(2):
    c = len(imgs[k])
    print(c)
    imagecount += c
    
    for maski,imgi,name in zip(masks_gt[k],imgs[k],img_names[k]):
        # print(name)
        c =  len(fastremap.unique(maski))
        cellcount += c
        
        if show:
            fig = plt.figure(figsize=(10,10))
            p = utils.normalize99(imgi)
            img0 = np.stack((p,p,p),axis=2)
            outli = plot.outline_view(img0,maski)

            pic = np.hstack((img0,utils.rescale(outli),cmap(ncolor.label(maski))[:,:,:3]))
            plt.imshow(pic)
            plt.axis('off')
            plt.show()

imagecount, cellcount

## Check to make sure that there is no duplication 

In [ ]:
# A = ['a','b','c']
# B = ['c']
A = img_names[0]
B = img_names[1]
[x for x in A if x in B]

### Export images, masks, and thumbnails

In [ ]:
from PIL import Image
import tifffile
from omnipose.utils import rescale, format_labels
offset = [0,len(imgs[0])]
# cmap2 = mpl.cm.get_cmap('gray')
# cmap2 = mpl.cm.get_cmap('magma')

clean = 0
save_ncolor = 0
save_color = 0
display = 0
mrcnn = 0 # export to mrcnn folder
matlab = 1 # export for morphometrics and supersegger processing

# get all the jpg files from the current folder
for k in range(2):
    # for f,im in zip(img_names[k],imgs[k]):
    for i,im,mask in zip(range(len(imgs[k])),imgs[k],masks_gt[k]):
        # name = 'img'+str(i+offset[k])
        # name = '%03d' % (i+offset[k],) + '_img'
        name = '%03d' % (i,) 
        
        if clean:
            #save a copy with new naming convention        
            savedir = basedir[k]+'_website'
            io.check_dir(savedir)
            io.imsave(os.path.join(savedir,name+'_img.tif'),im)  
            io.imsave(os.path.join(savedir,name+'_masks.tif'),mask)    

            # convert to thumbnail image
            im = (omnipose.utils.normalize99(im)*255).astype(np.uint8)
            im = Image.fromarray(im)
            im.thumbnail((128, 128), Image.ANTIALIAS)
            savedir = basedir[k]+'_website_thumbnails'
            io.check_dir(savedir)
            # im.save(os.path.join(savedir,getname(f)+'_thumbnail.tif'))
            im.save(os.path.join(savedir,name+'_thumbnail.tif'))
        
        #Also save an ncolor version
        if save_ncolor:
            pic = cmap(ncolor.label(mask))
            # pic[:,:,-1] = mask>0 #transparency, not needed
            savedir = basedir[k]+'_ncolor'
            io.check_dir(savedir)
            skimage.io.imsave(os.path.join(savedir,name+'_ncolor_masks.tif'),(pic*(2**8-1)).astype(np.uint8))
        
        # can also save rescaled version of label matrix
        if save_color:
            cmap2 = ListedColormap([color for color in utils.sinebow(mask.max()).values()])
            pic = cmap2(omnipose.utils.rescale(mask))
            pic[:,:,-1] = mask>0 #transparency
            savedir = basedir[k]+'_rescaled_sinebow'
            io.check_dir(savedir)
            skimage.io.imsave(os.path.join(savedir,name+'_masks_rescaled.tif'),(pic*(2**8-1)).astype(np.uint8))
            
        if display:
            fig = plt.figure(figsize=[4]*2)
            plt.imshow(np.hstack((utils.rescale(im),ncolor.label(mask))))
            plt.axis('off')
            plt.show()
            
        if mrcnn:
            name = getname(mask_names[k][i]) # use original names for this export 
            savedir = os.path.join('/home/kcutler/DataDrive/omnipose_all/RGB_for_mrcnn',subset[k])
            io.check_dir(savedir)
            # io.imsave(os.path.join(savedir,name+'_img.tif'),np.stack([im]*3))  
            # io.imsave(os.path.join(savedir,name+'_masks.tif'),mask)    
            
            mgt = omnipose.utils.format_labels(mask,clean=True)
            tifffile.imsave(os.path.join(savedir,name+'_masks.tif'),mgt.astype('int64'))
            
            img = np.repeat((rescale(im)*(2**16-1)).astype('uint16')[:,:,np.newaxis],repeats=3,axis=-1) # must use rgb images
            tifffile.imsave(os.path.join(savedir,name+'.tif'),img) 

            
        if matlab and subset[k]=='test':
            name = getname(mask_names[k][i]) # use original names for this export 
            savedir = os.path.join('/home/kcutler/DataDrive/omnipose_all/matlab')
            io.check_dir(savedir)
#             
            img = (rescale(im)*(2**16-1)).astype('uint16')
            tifffile.imsave(os.path.join(savedir,name+'.tif'),img) 

In [ ]:
(np.stack([im]*3)).shape

In [ ]:
getname(mask_names[0])

In [ ]:
mask_names[0][0]

In [ ]:
t = io.imread('/home/kcutler/DataDrive/omnipose_all/morphometrics_eval/5I_crop_ensemble_1_Gparent.tif')

In [ ]:
plt.imshow(t)